# Session 3

## X-ray source parameters ![gVXR](img/gvxr_logo.png)

## Authors: Ben Thorpe, Iwan Mitchel and Franck Vidal

(version 1.0, 22 Sep 2022)

# Aims of this session

- Create X-ray sources of different shapes:
    - Parallel beam (e.g. synchrotron);
    - Cone-beam (e.g. X-ray tube);
        - Ideal case, i.e. infinitesimal point source;
        - Focal spot, i.e. more realistic.
- Visualise the source shape;
- Set the beam spectrum:
    - Monochromatic;
    - Polychromatic:
        - By hand;
        - Using a text file;
        - Using [xpecgen](https://github.com/Dih5/xpecgen) (a python package to calculate x-ray spectra generated in tungsten anodes using the model of [Med. Phys. 43, 4655.](https://doi.org/10.1118/1.4955120)).

# Task 1: Initialise the simulation environment

At this stage, you should be able to set up the simulation environment. For now, you must:

1. Import the packages 
    - `from gvxrPython3 import gvxr # Simulate X-ray images`
    - `from gvxrPython3.utils import visualise` # Visualise the 3D environment if k3D is supported

1. Create the OpenGL context with `gvxr.createWindow`
2. Set the detector at position () in mm with `gvxr.setDetectorPosition`
3. Set the detector up vector as () with `gvxr.setDetectorUpVector`
4. Set the number of pixels: 600 along both directions with `gvxr.setDetectorNumberOfPixels`
5. Set the pixel spacing:  with `gvxr.setDetectorPixelSize`
6. Load the following samples

| Label | fname | Material type | Material | Density |
|-------|-------|---------------|----------|---------|
| Matrix | input_data/plate_sphere_inserts/matrix.stl | Compound | H2O | 1g.cm3 |

The functions you must use are:
-  ``
-  ``
-  ``
-  ``
-  ``

## Step 1 of Task 1: Import the packages

In [9]:
from gvxrPython3 import gvxr
from gvxrPython3.utils import visualise



Speckpy is not install, you won't be able to load a beam spectrum using Speckpy
SimpleGVXR 2.0.2 (2022-10-05T09:09:48) [Compiler: GNU g++] on Linux
gVirtualXRay core library (gvxr) 2.0.2 (2022-10-05T09:09:48) [Compiler: GNU g++] on Linux


## Step 3 of Task 1: Create the OpenGL context

In [8]:
window_id = 0
visible = False
opengl_major_version = 4
opengl_minor_version = 5
backend = "EGL"
gvxr.createWindow(window_id, visible, backend, opengl_major_version, opengl_minor_version);

Wed Oct  5 11:22:27 2022 ---- Create window (ID: 0)
Wed Oct  5 11:22:27 2022 ---- Query the number of EGL devices
Wed Oct  5 11:22:27 2022 ---- Success
Wed Oct  5 11:22:27 2022 ---- Detected 3 EGL devices.
Wed Oct  5 11:22:27 2022 ---- Print the details here of every EGL device.
Wed Oct  5 11:22:27 2022 ---- Success
Wed Oct  5 11:22:27 2022 ---- Device 1/3:
Wed Oct  5 11:22:27 2022 ---- 	Device Extensions: EGL_NV_device_cuda EGL_EXT_device_drm EGL_EXT_device_drm_render_node EGL_EXT_device_persistent_id
Wed Oct  5 11:22:27 2022 ---- 	Device vendor: NVIDIA
Wed Oct  5 11:22:27 2022 ---- 	EGL DRM device file: /dev/dri/card0
Wed Oct  5 11:22:27 2022 ---- Device 2/3:
Wed Oct  5 11:22:27 2022 ---- 	Device Extensions: EGL_EXT_device_drm
Wed Oct  5 11:22:27 2022 ---- 	Failed to retrieve device vendor.
Wed Oct  5 11:22:27 2022 ---- 	EGL DRM device file: /dev/dri/card0
Wed Oct  5 11:22:27 2022 ---- Device 3/3:
Wed Oct  5 11:22:27 2022 ---- 	Device Extensions: EGL_MESA_device_software
Wed Oct  5 1

## Step 3 of Task 1: Set the detector position

In [ ]:
gvxr.setDetectorNumberOfPixels(640,320)
gvxr.setDetectorPixelSize(0.5,0.5,"mm")
gvxr.setDetectorPosition(0.0,10.0,0.0,"cm")
gvxr.setDetectorUpVector(0,0,-1)

# X-ray spectrum

The simulation software below makes it possible to calculate spectra as generated by the [X-ray generators](https://www.creatis.insa-lyon.fr/~letang/x-ray-book/instrumentation.html#x-ray-production) by specifying:
- the high voltage U of electrons acceleration,
- the thickness and the nature of the plate inserted.

Certain quantities associated with the spectra are calculated at output in particular
- the average energy of the photons of the spectrum,
- the half-value layer (in mm of Al).

In [3]:
%matplotlib inline
import ipywidgets as widgets
import utilities
widgets.interact_manual(utilities.spectrum,E0=widgets.IntText(value=100,step=1,description="High voltage (kV):",disabled=False,style={'description_width': 'initial'}),Mat_Z=widgets.Dropdown(options=[('Water',-1),('Aluminium',13),('Copper',29),('Lead',82)],value=-1,layout={'width': 'max-content'},description='Material:',style={'description_width': 'initial'}),Mat_X=widgets.FloatText(value=0.,step=0.01,description="Thickness (mm):",disabled=False,style={'description_width': 'initial'}));


interactive(children=(IntText(value=100, description='High voltage (kV):', style=DescriptionStyle(description_…